<a href="https://colab.research.google.com/github/niccronc/TV-Show-Recommender/blob/master/IMDb_tv_show_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path='/content/drive/My Drive/Colab Notebooks/data/IMDb'

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv(path+'/storyline.csv')
df.head()

,Name,Genres,Score,Votes,Storyline
0,The Umbrella Academy,"['Action', 'Adventure', 'Comedy']",8.0,"114,948","On the same day in October 1989, forty-thr..."
1,Dark,"['Crime', 'Drama', 'Mystery']",8.8,"237,507",The first German production from the world...
2,Yellowstone,"['Drama', 'Western']",8.5,"19,008","Yellowstone follows the Dutton family, led..."
3,Perry Mason,"['Crime', 'Drama']",7.4,"5,404","Set in 1932 Los Angeles, the series focuse..."
4,Game of Thrones,"['Action', 'Adventure', 'Drama']",9.3,"1,696,047","In the mythical continent of Westeros, sev..."


In [ ]:
len(df)

9649

In [ ]:
!pip install transformers

     |████████████████████████████████| 890kB 2.9MB/s 
     |████████████████████████████████| 890kB 15.5MB/s 
     |████████████████████████████████| 3.0MB 17.4MB/s 
     |████████████████████████████████| 1.1MB 25.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=116f9c58733de625735a22ac840118606e4cef488384e29ace89b96f027914c4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)

In [ ]:
def get_vector(sentence):
  inputs = tokenizer(sentence, return_tensors="pt")
  outputs = model(**inputs)
  penultimate = outputs[2][11]
  pooler = penultimate[:, 0, :]
  return pooler.squeeze().detach().numpy()

In [ ]:
def get_vector_averaged(sentence):
  inputs = tokenizer(sentence, return_tensors="pt")
  outputs = model(**inputs)
  penultimate = outputs[2][11]
  average_pooler = torch.mean(penultimate,1)
  return average_pooler.squeeze().detach().numpy()

In [ ]:
def add_Bert_column(row):
  sentence = row['Storyline']
  row['Bert vector']=get_vector(sentence)
  return row

In [ ]:
def add_Bert_column_averaged(row):
  sentence = row['Storyline']
  row['Bert vector']=get_vector_averaged(sentence)
  return row

In [ ]:
sample_df=df.sample(n=5)
sample_df.head()

,Name,Genres,Score,Votes,Storyline
8290,Szomszédok,"['Drama', 'Family', 'Romance']",6.6,767,The everyday life of four families at a ho...
605,Frontier,"['Adventure', 'Drama', 'History']",7.2,"15,831","Follows Declan Harp, a half-Irish/half Cre..."
3539,The Angry Video Game Nerd,"['Comedy', 'Talk-Show']",8.5,"13,798",A series about a serious video game fan wh...
5761,Cobra,"['Action', 'Adventure']",6.5,455,When the traditional judicial system fails...
6527,Bitchin' Rides,['Reality-TV'],7.5,262,Dave Kindig and his crew in his Salt Lake ...


In [ ]:
sample_df.apply(add_Bert_column, axis=1)

,Name,Genres,Score,Votes,Storyline,Bert vector
8290,Szomszédok,"['Drama', 'Family', 'Romance']",6.6,767,The everyday life of four families at a ho...,"[-0.5061708, -0.21353494, -0.43998682, -0.3419..."
605,Frontier,"['Adventure', 'Drama', 'History']",7.2,"15,831","Follows Declan Harp, a half-Irish/half Cre...","[-0.63864297, -0.67690116, -0.40990752, 0.2392..."
3539,The Angry Video Game Nerd,"['Comedy', 'Talk-Show']",8.5,"13,798",A series about a serious video game fan wh...,"[-0.07893824, -0.79153925, -0.23967892, -0.262..."
5761,Cobra,"['Action', 'Adventure']",6.5,455,When the traditional judicial system fails...,"[-0.3917976, -0.676937, -0.45623264, 0.3461451..."
6527,Bitchin' Rides,['Reality-TV'],7.5,262,Dave Kindig and his crew in his Salt Lake ...,"[-0.32568833, 0.47927758, -0.6824654, 0.126722..."


So our strategy works, but beware that there is some entry in the Storyline column which is not a string! Why? Perhaps it is empty?

In [ ]:
df_clean=df[df.Storyline.notnull()]

In [ ]:
sample_clean=df_clean.sample(n=50)
sample_clean

,Name,Genres,Score,Votes,Storyline
2060,The Honeymooners,"['Comedy', 'Family']",8.6,"4,532",Ralph Kramden is a New York bus driver who...
6338,The Chris Isaak Show,"['Comedy', 'Music']",7.8,621,"Chris plays himself as a low-key, nebbishy..."
6428,Baieti de Oras,['Comedy'],7.8,305,"Six friends, all with different lifestyles..."
5724,Night Visions,"['Fantasy', 'Horror', 'Mystery']",7.5,798,Night Visions is an anthology series simil...
3189,American Odyssey,"['Action', 'Drama', 'Thriller']",7.3,"5,885",Three families are torn apart when a stran...
5035,"Neon Joe, Werewolf Hunter","['Comedy', 'Horror', 'Sci-Fi']",7.2,778,Set in the pretend sleepy enclave of Garri...
6905,Strike Witches,"['Animation', 'Action', 'Comedy']",6.4,314,The story takes place in year 1944 and the...
547,Snowfall,"['Crime', 'Drama']",8.0,"9,296",A look at the early days of the crack coca...
5473,Granite Flats,['Drama'],8.0,950,When a mysterious object falls from the sk...
2022,DC Super Hero Girls,"['Animation', 'Action', 'Adventure']",7.0,510,Teenage versions of DC Comics superheroine...


In [ ]:
sample_berted = sample_clean.apply(add_Bert_column, axis=1)

In [ ]:
sample_berted

,Name,Genres,Score,Votes,Storyline,Bert vector
2060,The Honeymooners,"['Comedy', 'Family']",8.6,"4,532",Ralph Kramden is a New York bus driver who...,"[-0.69951004, -0.35813463, -0.62968767, 0.1170..."
6338,The Chris Isaak Show,"['Comedy', 'Music']",7.8,621,"Chris plays himself as a low-key, nebbishy...","[-0.54087603, -0.24936864, -0.4600258, -0.1614..."
6428,Baieti de Oras,['Comedy'],7.8,305,"Six friends, all with different lifestyles...","[-0.4404717, -0.06990207, -0.593798, -0.182721..."
5724,Night Visions,"['Fantasy', 'Horror', 'Mystery']",7.5,798,Night Visions is an anthology series simil...,"[-0.32406402, -0.7709435, -0.072052374, -0.081..."
3189,American Odyssey,"['Action', 'Drama', 'Thriller']",7.3,"5,885",Three families are torn apart when a stran...,"[-0.08233038, 0.22790723, 0.10217152, -0.07743..."
5035,"Neon Joe, Werewolf Hunter","['Comedy', 'Horror', 'Sci-Fi']",7.2,778,Set in the pretend sleepy enclave of Garri...,"[-0.23787402, -0.4893779, 0.2309962, -0.825220..."
6905,Strike Witches,"['Animation', 'Action', 'Comedy']",6.4,314,The story takes place in year 1944 and the...,"[-0.61902875, -0.69865894, -0.052179985, -0.40..."
547,Snowfall,"['Crime', 'Drama']",8.0,"9,296",A look at the early days of the crack coca...,"[-0.18126571, -0.7698967, -0.54964757, -0.1772..."
5473,Granite Flats,['Drama'],8.0,950,When a mysterious object falls from the sk...,"[-0.12788418, -0.06497336, 0.020154394, 0.1575..."
2022,DC Super Hero Girls,"['Animation', 'Action', 'Adventure']",7.0,510,Teenage versions of DC Comics superheroine...,"[-0.74581796, -0.39093974, -0.724196, 1.010443..."


In [ ]:
df_berted=df_clean.apply(add_Bert_column, axis=1)

In [ ]:
df_berted_averaged=df_clean.apply(add_Bert_column_averaged, axis=1)

In [ ]:
df_berted_averaged.head()

,Name,Genres,Score,Votes,Storyline,Bert vector
0,The Umbrella Academy,"['Action', 'Adventure', 'Comedy']",8.0,"114,948","On the same day in October 1989, forty-thr...","[-0.2231853, -0.058823504, 0.38834637, -0.2301..."
1,Dark,"['Crime', 'Drama', 'Mystery']",8.8,"237,507",The first German production from the world...,"[-0.3973119, -0.2140088, 0.66914797, -0.238113..."
2,Yellowstone,"['Drama', 'Western']",8.5,"19,008","Yellowstone follows the Dutton family, led...","[-0.104803674, -0.03705466, 0.14606862, -0.060..."
3,Perry Mason,"['Crime', 'Drama']",7.4,"5,404","Set in 1932 Los Angeles, the series focuse...","[-0.27238184, -0.1082099, -0.040502563, -0.259..."
4,Game of Thrones,"['Action', 'Adventure', 'Drama']",9.3,"1,696,047","In the mythical continent of Westeros, sev...","[-0.28576553, 0.03967202, 0.32915372, -0.16211..."


In [ ]:
type(df_berted['Bert vector'].iloc[4][3])

numpy.float32

In [ ]:
df_berted.to_csv(path+'/df_berted.csv', index=False)

In [ ]:
df_berted=pd.read_csv(path+'/df_berted.csv')
df_berted.head()

,Name,Genres,Score,Votes,Storyline,Bert vector
0,The Umbrella Academy,"['Action', 'Adventure', 'Comedy']",8.0,"114,948","On the same day in October 1989, forty-thr...",[-6.59081221e-01 -2.34883964e-01 -6.98900446e-...
1,Dark,"['Crime', 'Drama', 'Mystery']",8.8,"237,507",The first German production from the world...,[-1.96405485e-01 -6.05088592e-01 1.08784020e-...
2,Yellowstone,"['Drama', 'Western']",8.5,"19,008","Yellowstone follows the Dutton family, led...",[-4.72666889e-01 -2.96435505e-01 -2.20258497e-...
3,Perry Mason,"['Crime', 'Drama']",7.4,"5,404","Set in 1932 Los Angeles, the series focuse...",[-3.67320359e-01 -9.49564695e-01 2.84631640e-...
4,Game of Thrones,"['Action', 'Adventure', 'Drama']",9.3,"1,696,047","In the mythical continent of Westeros, sev...",[-5.92308104e-01 -2.74694830e-01 1.04733519e-...


In [ ]:
def cosine_similarity(v,w):
  return np.dot(v,w) / ( np.linalg.norm(v) * np.linalg.norm(w) )

In [ ]:
def array_to_list(row):
  row['Bert_list']=row['Bert vector'].tolist()
  return row

In [ ]:
df_berted_averaged = df_berted_averaged.apply(array_to_list, axis=1)

In [ ]:
sample_berted.head()

,Name,Genres,Score,Votes,Storyline,Bert vector,Bert_list
2060,The Honeymooners,"['Comedy', 'Family']",8.6,"4,532",Ralph Kramden is a New York bus driver who...,"[-0.69951004, -0.35813463, -0.62968767, 0.1170...","[-0.6995100378990173, -0.3581346273422241, -0...."
6338,The Chris Isaak Show,"['Comedy', 'Music']",7.8,621,"Chris plays himself as a low-key, nebbishy...","[-0.54087603, -0.24936864, -0.4600258, -0.1614...","[-0.540876030921936, -0.24936863780021667, -0...."
6428,Baieti de Oras,['Comedy'],7.8,305,"Six friends, all with different lifestyles...","[-0.4404717, -0.06990207, -0.593798, -0.182721...","[-0.44047170877456665, -0.06990206986665726, -..."
5724,Night Visions,"['Fantasy', 'Horror', 'Mystery']",7.5,798,Night Visions is an anthology series simil...,"[-0.32406402, -0.7709435, -0.072052374, -0.081...","[-0.3240640163421631, -0.7709435224533081, -0...."
3189,American Odyssey,"['Action', 'Drama', 'Thriller']",7.3,"5,885",Three families are torn apart when a stran...,"[-0.08233038, 0.22790723, 0.10217152, -0.07743...","[-0.0823303833603859, 0.2279072254896164, 0.10..."


In [ ]:
df_berted = df_berted.apply(array_to_list, axis=1)

In [ ]:
v_1=sample_berted.iloc[1]['Bert_list']
v_2=sample_berted.iloc[2]['Bert_list']

In [ ]:
cosine_similarity(v_1,v_2)

0.6996046357077184

In [84]:
def sentence_comparison(sentence):
  """ Input: user-provided sentence. Output: cosine-similarity value"""
  sentence_vector=get_vector_averaged(sentence).tolist()
  def cos_simil(vector):
    return cosine_similarity(vector, sentence_vector)
  return cos_simil

In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 65kB/s 
     |████████████████████████████████| 204kB 36.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=1fafff38e4c154c61f8d8180cf9fa29191471b8c0ec34e3aa4dc080bb77e4a4a
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark import SparkContext, SparkConf
sc = SparkContext("local[*]", "pyspark_df")
print(sc.version)

3.0.1


In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
df_basic=df_berted[['Name','Bert vector']]

In [ ]:
df_basic.Bert_list=df_basic['Bert vector'].map(lambda x: x.tolist())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [ ]:
df_basic.iloc[4]['Bert vector'].tolist()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, ArrayType, DoubleType

In [ ]:
mySchema = StructType([ StructField("Name", StringType(), True)\
                       ,StructField("Bert_vector", ArrayType(elementType=DoubleType() ), True) ])

In [ ]:
df = sqlContext.createDataFrame(df_berted_averaged[['Name','Bert_list']],schema=mySchema)

In [ ]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Bert_vector: array (nullable = true)
 |    |-- element: double (containsNull = true)



In [ ]:
df.show(2)

+--------------------+--------------------+
|                Name|         Bert_vector|
+--------------------+--------------------+
|The Umbrella Academy|[-0.2231853008270...|
|                Dark|[-0.3973118960857...|
+--------------------+--------------------+
only showing top 2 rows



In [88]:
test_sentence='Writer helps attractive New York detective solve crimes'

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
cos_similarity = udf(sentence_comparison(test_sentence), DoubleType())

In [89]:
sentence_comparison_func = sentence_comparison(test_sentence)

In [ ]:
df.withColumn('cosine_similarity',cos_similarity('Name')).show(5)
#.orderBy(df.cosine_similarity.desc()).collect(5)

In [ ]:
new_df

DataFrame[Name: string, Bert_vector: array<double>, cosine_similarity: double]

In [ ]:
df_rdd = df.rdd

In [ ]:
df_rdd.take(2)

[Row(Name='The Umbrella Academy', Bert_vector=[-0.22318530082702637, -0.05882350355386734, 0.3883463740348816, -0.2301371991634369, 0.47817468643188477, 0.05000431463122368, -0.09812726080417633, 0.3094232380390167, 0.5600964426994324, -0.1167328730225563, 0.11140818893909454, -0.31064730882644653, -0.015597139485180378, 0.7144033312797546, -0.09557406604290009, 0.5404767394065857, 0.17480722069740295, 0.20652316510677338, -0.3230994939804077, 0.6028876304626465, 0.48522430658340454, 0.06768546998500824, -0.3924400210380554, 1.0388994216918945, 0.31149986386299133, 0.32528552412986755, -0.11563627421855927, -0.26295286417007446, 0.06730690598487854, 0.4071703851222992, 0.37623947858810425, -0.3060328960418701, -0.0072305696085095406, -0.3829636573791504, -0.13504236936569214, -0.08164165169000626, 0.10307316482067108, 0.009611382149159908, 0.3052409887313843, 0.350017786026001, -0.1798165738582611, -0.3196595311164856, -0.0895063504576683, 0.18308457732200623, 0.17825080454349518, -0.2

In [90]:
new_rdd = df_rdd.map(lambda x : (x.Name, sentence_comparison_func(x.Bert_vector)))

In [91]:
new_rdd.sortBy(lambda x : - x[1]).collect()[:10]

[('Training Day', 0.8350765228795355),
 ('Murder City', 0.8303878763501701),
 ('Oficer', 0.8300519724762891),
 ('Conviction', 0.8287491046761024),
 ('Navarro', 0.8253441520640925),
 ('A Case for Two', 0.824259405182708),
 ('Against the Wall', 0.821796817965708),
 ('Çember', 0.8196834123352987),
 ('The Old Fox', 0.8194812171751826),
 ('Cold Case Files', 0.8189504844875398)]